In [207]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd
import numpy as np
import re
import json
import ast

functions = ['extract_title',
             'create_contact_json',
             'create_location_json',
             'extract_project_partners',
             'create_project_type_json',
             'create_project_description_json',
             'create_readiness_to_proceed_json',
             'create_budget_json',
             'create_contribution_to_irwmp_json',
             'create_affected_groundwater_basins_json',
             'create_affected_baneficial_uses_json',
             'create_planning_for_climate_change_json',
             'create_rsrc_management_strategies_json',
             'create_stakeholder_outreach_json',
             'create_environmental_compliance_json',
             'create_feasibility_doc_json',
             'create_schedule_json']
#              'create_eligibility_json']
ls = u"['Title','Contact','Location','Partners','Type','Description','Readiness To Proceed','Budget','Contribution to IRWMP objectives','Affected Groundwater Basins','Affected Beneficial Uses','Planning for Climate Change','Resource Management Strategies','Stakeholder Outreach and Involvement','Environmental Compliance','Feasibility Documentation','Schedule']"
columns = ast.literal_eval(ls)

In [208]:
def main():
    kLoginPage = "http://irwm.rmcwater.com/cv/login.php"
    kUsername =  "jlikhuva@stanford.edu"
    kPassword =  "stanford2017"
    tbody, questionnaire_list = extractPageHTML(kLoginPage, kUsername, kPassword)
#     create_preliminary_database(tbody)
    create_full_database(questionnaire_list)
    questionnaire_list_g = questionnaire_list

In [209]:
def create_preliminary_database(tbody):
    bsObj = generateBeautifulSoupObject(tbody) 
    row =   extractAllTableEntries(bsObj)
    index = []
    np_array_data = []
    for elem in row:
        data = extractDataFromRow(elem)
        index.append(data.pop(0))
        np_array_data.append(data)
    columns = ['Organization', 'Title', 'Functional Area', 'Grant Request', 'Total Cost']
    data = np.array(np_array_data)
    df = pd.DataFrame(data, index=index, columns=columns)
    df.to_excel('Coachella Valley Database.xlsx')

In [210]:
def non_ascii_rem(str):
    return ''.join([i if ord(i) < 128 else '' for i in str])

def split_join(str):
    trimmed_words = []
    words = str.split()
    for each in words:
        trimmed_words.append(each.strip())
    return " ".join(trimmed_words)

def create_full_database(questionnaire_list):
    df = pd.read_csv('./column_titles')
    index = 0
    for questionnaire in questionnaire_list:
        bs_obj = generateBeautifulSoupObject(questionnaire)
        
        entries = []
        for i in xrange(1, 5):
            cur_tbody_id = 'tbdTab_' + str(i)
            cur_bs_obj = bs_obj.find("tbody", {"id": cur_tbody_id})
            entries += extractAllTableEntries(cur_bs_obj)
            
        
        for idx, entry in enumerate(entries):
            entries[idx] = split_join(non_ascii_rem(re.sub('\s+',' ',str(entry)).strip())) 
        
        features = {}
        for idx, function in enumerate(functions):
            cur_feature = columns[idx]
            cur_feature_value = globals()[function](entries)
            features[cur_feature] = cur_feature_value
            
        index_ = [index, ]
        index += 1
        new_df = pd.DataFrame(features, index=index_)
        df = pd.concat([df, new_df])
        
    df.to_excel('Database.xlsx')

In [211]:
def extract(values, index):
    return generateBeautifulSoupObject(str(values[index])).text

def create_json(values, positions_and_keys):
    json_creator = {}
    for key, value in positions_and_keys.iteritems():
        json_creator[key] = extract(values, value)
    return json.dumps(json_creator)

def extract_title(values):
    return extract(values, 1)
     
def extract_project_partners(values):
    return extract(values, 35)

In [212]:
def create_contact_json(values):
    keys = ['name', 'title', 'agency', 'address_1', 'address_2',
           'city_state_zip', 'phone', 'email', 'share_perm']
    json_creator = {}
    to_add = 4
    for idx, each in enumerate(keys):
        cur_pos = idx + to_add
        cur_v = extract(values, cur_pos)
        json_creator[each] = cur_v
        to_add += 1
    return json.dumps(json_creator)

In [213]:
def create_location_json(values):
    keys = {'description' : 23, 'acreage' : 25, 'latitude' : 28, 'longitude': 29}
    return create_json(values, keys)

In [214]:
def create_project_type_json(values):
    keys = {'primary_functional_area':41, "other_functional_area":43,
           'primary_project_type':45}
    return create_json(values, keys)

In [215]:
def create_project_description_json(values):
    keys = {'summary':51,'detailed_description':53,'linkages_with_other_projects':55,
            'local/regional plans that list this project':57,
            'how_it_creates_new_water':59}
    return create_json(values, keys)

In [216]:
def create_readiness_to_proceed_json(values):
    keys = {'ready_for_prop84':62,'provides direct water benefits to DACs':65,
           'long term drought preparedness':67, 'quantifiable benefits':69}
    return create_json(values, keys)

In [217]:
def create_budget_json(values):
    keys = {'cost':211,'requested funds':213,'local match':215,'match type':217, 
     'Annual O&M cost':219,'O&M type':221}
    return create_json(values, keys)

In [218]:
def create_contribution_to_irwmp_json(values):
    keys = {}
    objective_list = [chr(i) for i in range(65, 78)]
    start, end = 78, 102
    pos = 0
    for i in range(start, end, 2):
        keys[objective_list[pos]] = pos
        pos += 1
    return create_json(values, keys)


In [219]:
def create_affected_groundwater_basins_json(values):
    keys = {'names of basins':104,'How':106}
    return create_json(values, keys)

In [220]:
def create_affected_baneficial_uses_json(values):
    keys = {'uses affected':109, 'How':111}
    return create_json(values, keys)

In [221]:
def create_planning_for_climate_change_json(values):
    keys = {'adapt':114, 'mitigate contribution':116}
    return create_json(values, keys)

In [222]:
def create_rsrc_management_strategies_json(values):
    start, end = 120, 176
    keys_list = [str(k) for k in range(30)]
    keys = {}
    i = 0
    for pos in range(start, end+1, 2):
        keys[keys_list[i]] = pos
        i += 1
    return create_json(values, keys)

In [223]:
def create_stakeholder_outreach_json(values):
    keys = {'public outreach methods':178,'methods to reach DACs':180}
    return create_json(values, keys)

In [224]:
def create_environmental_compliance_json(values):
    keys = {'permits':186,'CEQA/NEPA':188}
    return create_json(values, keys)

In [225]:
def create_feasibility_doc_json(values):
    keys = {'feasibility studies':191, 'project need':193}
    return create_json(values, keys)

In [226]:
def create_schedule_json(values):
    keys = {'planning_start':196,'planning_end':197,'design_start':200,'design_end':201,
           'construction_start':205,'construction_end':206}
    return create_json(values, keys)

In [227]:
# def create_eligibility_json(values):
#     pass
#     return create_json(values, keys)

In [228]:
def extractPageHTML(url, username, password):
    browser = webdriver.Chrome("./chromedriver")
    browser.get(url)
    username_field = browser.find_element_by_name('username')
    password_field = browser.find_element_by_name('password')
    submit_button  = browser.find_element_by_name('Login')
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()
    all_projects_tab = browser.find_element_by_link_text('All Submitted Projects')
    all_projects_tab.click()
    
    all_projects_tbody = None
    try:
        all_projects_tbody = browser.find_element_by_id('tbdy_track_all')
    except:
        browser.switch_to.alert.dismiss();
        sleep(1)
        all_projects_tbody = browser.find_element_by_id('tbdy_track_all')
        
    form_buttons = browser.find_elements_by_xpath('//img[@src="images/frmView.png"]')
    return all_projects_tbody.get_attribute('innerHTML'), extract_questionnaire_inner_html(form_buttons, browser)

def extract_questionnaire_inner_html(form_buttons, browser):
    questionnaire_list = []
    for button in form_buttons:
        button.click()
    browser.close()
    for window in browser.window_handles:
        browser.switch_to_window(window)
        innerHTML = browser.execute_script("return document.body.innerHTML")
        questionnaire_list.append(innerHTML)
        browser.close()
        
    return questionnaire_list

In [229]:
def generateBeautifulSoupObject(html):
    try:
        bsObject = bs(html, 'html.parser')  # from the bs4 3rd part library
    except AttributeError as e:
        return None
    return bsObject

In [230]:
def extractAllTableEntries(bsobj):
    rows = bsobj.find_all('td') 
    return rows

In [231]:
def extractDataFromRow(row):
    row = re.sub('\s+',' ',str(row)).strip()
    start_end_tuples = [(m.start(), m.end()) for m in re.finditer('</td>', row)]
    positions = []
    positions.append((0, start_end_tuples[0][0]))
    for index, tup, in enumerate(start_end_tuples):
        if index+1 == len(start_end_tuples):
            break
        positions.append((start_end_tuples[index][1], start_end_tuples[index+1][0]))
    elems = [row[pos[0]:pos[1]] for pos in positions]
    to_remove = [0, 1, 1]
    for v in to_remove:
        del elems[v]
    clean = [elem[elem.rfind('>')+1:].strip().strip('\xc2\xa0') for elem in elems]
    return clean

In [232]:
main()